# NLP Deep Dive: RNNs

Language model verilen text için sıradaki kelimenin ne olacağını tahmin etmeye çalışan bir modeldir. Bu task'e **self-supervised learning** denir çünkü model yine supervised bir learning gerçekleştiriyor ancak label'ları biz manuel olarak vermiyoruz, biz sadece text corpus'u veriyoruz, zaten otomatik olarak labeled gibi oluyor. Bir language modelin verilen kelimeler için sıradaki kelimeye tahmin edebiliyor olması, ilgili dili anlıyor olmasını gerektirir, zaten language modelle asıl yapılmak istenen budur. Dilin temellerini anlayan bir model eğittikten sonra bu modeli özelleşmiş bir amaç için eğitiriz.

> jargon: Self-supervised learning: Training a model using labels that are embedded in the independent variable, rather than requiring external labels. For instance, training a model to predict the next word in a text.

Daha önce intro amacıyla kullandığımız language model IMDB reviews'i classify etmek için özelleşmişti, bu model ilk etapta Wikipedia datası ile pretrained edildi, daha sonra bu pretrained modeli IMBD reviews için fine-tune ettiğimizde çok iyi sonuçlar elde edebiliyoruz.

The Wikipedia English is slightly different from the IMDb English, so instead of jumping directly to the classifier, we could fine-tune our pretrained language model to the IMDb corpus and then use *that* as the base for our classifier.

Pretrained model temel dil kurallarını anlamış olsa dahi, asıl task'in olduğu corpus'un stili bundan farklı olacaktır örneğin IMDB dataset'i için daha informal bir language kullanılacaktır, ayrıca task'e göre dilin için teknik terimler veya bilinmesi gereken özel isimler olabilir. IMDB dataset'i için bir çok film, oyuncu, yönetmen ismi olacak. Modelimizin bu kelimeleri daha iyi anlayabilmesi için pre-trained language modeli fine-tune etmek isteriz.

Daha önce gördük ki, kolayca pretrained bir English language model'i indirebiliriz ve bu model ile kendi task'imiz için state-of-the-art results elde edebiliriz. Bunun yanında başka diller için de pre-trained modeller bulmak mümkün olacaktır. O halde bir language model'in nasıl eğitildiğini öğrenmek gereksiz mi?

Elbette ki değil, bir kere zaten yapılan uygulamanın arkaplanda nasıl çalıştığımı anlıyor olmak her zaman avantajlıdır. Bunun dışında pretrained language modeli fine-tune ettiğimizde performansı artırıyoruz, yalnız karışmasın, şunu demek istiyoruz normalde pre-trained language modeli alıp direkt olarak task üzerinde classifier olarak fine-tune edebiliriz. Burada bundan bahsetmiyoruz, pre-trained modeli alıp yine language model olarak tüm task corpus'u üzerinde bir fine-tune ediyoruz. Daha sonra modeli manipüle edip (muhtemeneln head kısmını kesip, yerine task için gerekli head'i koyarak) classifier olarak tekrar fine-tune ediyoruz. Böylece model performansı bariz olarak artıyor.

Örneğin IMDB sentiment analysis task'i için dataset 50k unlabeled movie reviews içeriyor,25k training ve 25k validation'ın yanında 50k unlabeled data totalde 100k movie reviews yapıyor. Biz tüm bu 100k datayı pretrained language modeli fine-tune etmek için kullanabiliriz, sonuça elimizde özellikle movie reviews için next word'ü tahmin etmeye çalışan bir language model olacak. Daha sonra final adımında ise bu özelleşmiş language modeli de alıp bir classifier olarak, 25k training data üzerinde tekrar fine-tune ediyoruz.

Bu yönteme **Universal Language Model Fine-tuning (ULMFit)** deriz. The [paper](https://arxiv.org/abs/1801.06146) showed that this extra stage of fine-tuning of the language model, prior to transfer learning to a classification task, resulted in significantly better predictions.

Sonuçta Universal Language Model Fine-tuning yöntemi için üç temel adım söz konusu:

<img alt="Diagram of the ULMFiT process" width="700" caption="The ULMFiT process" id="ulmfit_process" src="../images/att_00027.png">

Şimdi neural network'ü bu problem için nasıl kullanabiliriz ona odaklanacağız. 

## Text Preprocessing

Daha önce öğrendiklerimizi kullanarak bir language model'i nasıl üreteceğiz? Cümleler farklı uzunluklarda olabilir, documents çok uzun olabilir, o halde neural networks kullanarak next word'ü nasıl tahmin edebiliriz? Buna bakalım.

Daha önce neural networks için categorical variables'ın nasıl inputs olarak kullanılabileceğine bakmıştık. Tek bir categorical variable için yakşlaşımımız şöyleydi:

1. Make a list of all possible levels of that categorical variable (we'll call this list the **vocab**).
1. Replace each level with its index in the vocab.
1. Create an embedding matrix for this containing a row for each level (i.e., for each item of the vocab).
1. Use this embedding matrix as the first layer of a neural network. (A dedicated embedding matrix can take as inputs the raw vocab indexes created in step 2; this is equivalent to but faster and more efficient than a matrix that takes as input one-hot-encoded vectors representing the indexes.)

Aynı yaklaşımı yani embeddings yaklaşımını text için de kullanabiliriz. Burada yeni olan **sequence** fikri olacak. İlk olarak datasetimizdeki tüm documents'i çok uzun tek bir string olacak şekilde concatenate ederiz. Daha sonra bu uzun string'i kelimelere böleriz, elimizde çok sayıda kelime (veya "token") olacak. 

Our independent variable will be the sequence of words starting with the first word in our very long list and ending with the second to last, and our dependent variable will be the sequence of words starting with the second word and ending with the last word. 

Yani anladığım kadarıyla input olarak ilk satır ilk kelime olacak ilk target da ikinci kelime. Bu şekilde ikinci input satırı, ilk iki kelime olacak, bunun target'ı ise 3. kelime olacak. Böyle böyle son input son kelime hariç tüm text'i içerecek, son target ise corpus'un son kelimesi olacak. Bu bana biraz verimsiz geldi ama anladığım kadarıyla yöntem bu.


Our vocab will consist of a mix of common words that are already in the vocabulary of our pretrained model and new words specific to our corpus (cinematographic terms or actors names, for instance). Our embedding matrix will be built accordingly: for words that are in the vocabulary of our pretrained model, we will take the corresponding row in the embedding matrix of the pretrained model; but for new words we won't have anything, so we will just initialize the corresponding row with a random vector.

Each of the steps necessary to create a language model has jargon associated with it from the world of natural language processing, and fastai and PyTorch classes available to help. The steps are:

- Tokenization:: Convert the text into a list of words (or characters, or substrings, depending on the granularity of your model)
- Numericalization:: Make a list of all of the unique words that appear (the vocab), and convert each word into a number, by looking up its index in the vocab
- Language model data loader creation:: fastai provides an `LMDataLoader` class which automatically handles creating a dependent variable that is offset from the independent variable by one token. It also handles some important details, such as how to shuffle the training data in such a way that the dependent and independent variables maintain their structure as required
- Language model creation:: We need a special kind of model that does something we haven't seen before: handles input lists which could be arbitrarily big or small. There are a number of ways to do this; in this chapter we will be using a *recurrent neural network* (RNN). We will get to the details of these RNNs in the <<chapter_nlp_dive>>, but for now, you can think of it as just another deep neural network.

Let's take a look at how each step works in detail.

### Tokenization

Tokenization adımında, text'i list of words'e çeviririz dedik. Peki noktalama işaretlerini ne yapıyoruz, veya "don't" gibi kelimerle nasıl başa çıkıyoruz, tek bir kelime olarak mı alıyoruz yoksa iki kelime olarak mı? Uzun medical veya teknik terimler için ne yapıyoruz? Bunları kelime kelime ayırmalı mıyız? Tireli kelimeler? Vesaire vesaire...

Bunların kesin bir cevabı yok, tokenization için tek bir yaklaşım söz konusu değil, üç temel yaklaşımdan söz edebiliriz.

Because there is no one correct answer to these questions, there is no one approach to tokenization. There are three main approaches:

- Word-based:: Split a sentence on spaces, as well as applying language-specific rules to try to separate parts of meaning even when there are no spaces (such as turning "don't" into "do n't"). Generally, punctuation marks are also split into separate tokens.
- Subword based:: Split words into smaller parts, based on the most commonly occurring substrings. For instance, "occasion" might be tokenized as "o c ca sion."
- Character-based:: Split a sentence into its individual characters.

We'll be looking at word and subword tokenization here, and we'll leave character-based tokenization for you to implement in the questionnaire at the end of this chapter.

> jargon: token: One element of a list created by the tokenization process. It could be a word, part of a word (a _subword_), or a single character.

### Word Tokenization with fastai

Fastai'ın kendi tokenizer'ı yoktur, onun yerine external libraries'in tokenizer'larını kullanabilmek için bir arayüz sağlar. Tokenization aktif research alanlarından birisi ve sürekli olarak yeni tokenizer'lar oluşturuluyor, fastai'ın default tokenizer'ı de değişecektir ancak API ve options çok değişmeyecektir, fastai altında yatan teknoloji değişse de sabit bir API sağlamaya çalışır.

Şimdi tokenization process'ini IMDB dataset'i ile deneyelim:

In [45]:
from fastai.text.all import *
from IPython.display import display,HTML

path = untar_data(URLs.IMDB)

Tokenizer'ı kullanmak için öncelikle tüm text files'ı grab etmemiz gerek. Daha önce nasıl ki `get_image_files` ile bir path içerisindeki tüm image files'ı grab ettik, şimdi de `get_text_files` ile bir path içerisindeki tüm text files'ı grab edeceğiz. Opsiyonel olarak fonskiyona `folders` geçirerek, text arasını spesifik subfolders ile sınırlandırabiliriz. 

Örneğin aşağıdaki path içerisindeki spesifik folder'ları tüm text files alınsın dedik.

In [4]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

File içerisinde tutulan şey .txt dosyalarınının path'leri oluyor, aynı images objelerinin path'leri gibi. Her bir .txt dosyası içerisinde bir review tutuluyor.

In [5]:
files

(#100000) [Path('/storage/data/imdb/unsup/23490_0.txt'),Path('/storage/data/imdb/unsup/46293_0.txt'),Path('/storage/data/imdb/unsup/44643_0.txt'),Path('/storage/data/imdb/unsup/18801_0.txt'),Path('/storage/data/imdb/unsup/37510_0.txt'),Path('/storage/data/imdb/unsup/17529_0.txt'),Path('/storage/data/imdb/unsup/16722_0.txt'),Path('/storage/data/imdb/unsup/6943_0.txt'),Path('/storage/data/imdb/unsup/45937_0.txt'),Path('/storage/data/imdb/unsup/7388_0.txt')...]

In [7]:
files[0].open().read()

"The worst movie I've ever seen, hands down. It is ten times more a rip-off of Lake Placid than it is a sequel. Director David Flores clearly did not go to film school, and the way his cast delivers they're lines, you'd think they were learning English for the first time. Not even Cloris Leachman tries. The first Nintendo games had more convincing special effects. Needless to say I didn't make it to the end of Lake Placid 2, but you don't need to watch more than five minutes to know that this is the biggest waste of celluloid in modern film. Do not let your love of the original tempt you to try this, even if you know it's bad. It's a turkey, no not even a turkey, it's nothing."

İlgili .txt files içerisinden  ilk file alınıp, açılmış ve içeriği okunmuş. Yerden kazanmak amacıyla ilk 75 karakteri alınacak.

In [9]:
txt = files[0].open().read(); txt[:75]

"The worst movie I've ever seen, hands down. It is ten times more a rip-off "

Bu döküman hazırlanırken fastai'ın default olarak kullandığı tokenizer'ı **spaCy** isminde bir library kullanır. Bu tokenizer'ın URLs, individual words, etc. için sophisticated kuralları vardır. Biz doğrudan `SpacyTokenizer` kullanmak yerine 
`WordTokenizer` kullanılacak, şuanda WordTokenizer zaten SpacyTokenizer'ı point ediyor ama gelecekte bu değişebilir, WordTokenizer her zaman en güncel tokenizer'ı point edecek.

Ayrıca burada tokenleri göstermek için fastai's `coll_repr(collection, n)` fonksiyonu kullanılacak. Bu fonksiyon bir `collection`'ın ilk `n` item'ı ile birlikte collection uzunluğunu da gösterecek. Şunu dikkat et: fastai tokenizers'ı tokenize etmek için collection of documents alıyor yani direkt bir string veremiyoruz, tek bir string vereceksek de bunu bir list ile wrap edip öyle veriyoruz:

In [18]:
#Önce tokenizer oluşturuldu.
spacy = WordTokenizer()
#Sonra tokenizer içerisine tek bir review string'ini bir list ile wrap ederek verdik: [txt] şeklinde
toks = first(spacy([txt]))
#Şimdi token listesini görmek için coll_repr fonksiyonunu kullanıyoruz, ilk 30 tokene bakalım:
print(coll_repr(toks, 30))

(#156) ['The','worst','movie','I',"'ve",'ever','seen',',','hands','down','.','It','is','ten','times','more','a','rip','-','off','of','Lake','Placid','than','it','is','a','sequel','.','Director'...]


Gördüğümüz üzere, spaCy temelde verilen string'i kelimelere ve noktalama işaretlerini ayırıyor. Ancak bu işin sanıldığı kadar straight-forward olmadığına dikkat et. Örneğin "it's" kelimesini "it" ve "'s" olarak tokenize ettiğini görüyorsundur, bu mantıklı çünkü gerçejten de bu iki token de kendi başına ayrı kelimeler. 

Tokenization gerçekten incelik isteyen bir iş, neyse ki spaCy detaylarla bizim için ilgileniyor. Örneğin aşağıdaki örnek cümleye bakarsak "." işaretinin eğer cümle sonlandırmak için kullanılıyorsa ayrıldığına ancak sayı içerisinde veya kısaltmalarda ise ayrılmadığına dikkat et.

In [20]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

Sonuç olarak ben tokenizer'ı kullanarak verilen text için `yazım sırasıyla` tüm tokenleri elde edip bir liste içerisinde alıyorum. 

Spacy ismindeki WordTokenizer obejsini fastai Tokenizer objesine beslersek, yeni fonksiyonlar elde edebiliriz. Aşağıda bu yapıllmış, tkn objesini ekstra fonksiyonları olan bir tokenizer olarak düşünebiliriz.

Aşağıda görüldüğü gibi tkn objesine string'i list ile wrap etmeden de besleyebiliyoruz, ardından da first kullanamdan doğrudan coll_repr kullanabiliyor ve listeyi göreniliyoruz:

In [21]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#176) ['xxbos','xxmaj','the','worst','movie','xxmaj','i',"'ve",'ever','seen',',','hands','down','.','xxmaj','it','is','ten','times','more','a','rip','-','off','of','xxmaj','lake','xxmaj','placid','than','it'...]


Burada bazı tokenlerin "xx" ile başladığını dikkat et bu token'ler kelimelere karşılık gelmezler **special tokens** olarak adlandırılırlar.

Örneğin yukarıda txt'i tokenize ettiğimizde ilk token `xxbos` olarak elde edildi bu token yeni text'in başladığını ifade eden bir tokendir ("Beginning of Stream"). Bu token sayesinde modelin yeni bir text'e geçildiğini ve bundan önce söylenenleri unutması gerektiğini öğrenmesi mümkün olur.

Bu special tokens'i spaCy sağlamıyor, bunu fastai defaul olarak sağlıyor. Belirli kurallar sayesinde modelin text'in önemli kısımlarını anlaması mümkün kılınıyor. Bir başka deyişle, English dilideki sequence'ı simplify edilmiş tokenized bir dile çeviriyoruz, böylece amacımız modelin dili daha rahat kavrayabilmesini sağlamak.

Örneğin eğer "!!!!" gibi bir yazı ile karşılaşılırsa bu kurallar sayesinde bunu tek bir ! işaretinin yanında **repeated character** tokeni ve sonra da number 4 olarak ifade ederiz. Böylece modelin embedding matrix'ine tekrar edilen karakterlerin anlamını kavrayabilmesi için bir şans veririz. Böyle yapmasaydık farklı repetition sayıları için farklı farklı tokenler gerekcekti bu impractical.

Benzer şekilde büyük harfle başlayan bir kelime özel bir **capitalization token** yanında lowercase version of this word ile değiştirilecektir. Böylece kelimeler bir kez tokenize edilmiş olur, büyük harfle yazılan bir kelime için ayrı, küçük harf için ayrı bir token tutmamış oluruz, büyük harf kavramını ayrı bir token olarak devreye sokmuş oluruzç

Aşağıda karşılaşabileceğimiz üç temel token görünüyor:

- `xxbos`:: Indicates the beginning of a text (here, a review)
- `xxmaj`:: Indicates the next word begins with a capital (since we lowercased everything)
- `xxunk`:: Indicates the next word is unknown

Aşağıda default olarak kullanılan rule'ların listesini görebiliriz:

In [18]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

Her zamanki gibi istersek istediğimiz bir kuralın source code'una aşağıdaki gibi bakabiliriz:

```
??replace_rep
```

Her bir kuralın kısaca ne yaptığı aşağıda yazıyor:

- `fix_html`:: Replaces special HTML characters with a readable version (IMDb reviews have quite a few of these)
- `replace_rep`:: Replaces any character repeated three times or more with a special token for repetition (`xxrep`), the number of times it's repeated, then the character
- `replace_wrep`:: Replaces any word repeated three times or more with a special token for word repetition (`xxwrep`), the number of times it's repeated, then the word
- `spec_add_spaces`:: Adds spaces around / and #
- `rm_useless_spaces`:: Removes all repetitions of the space character
- `replace_all_caps`:: Lowercases a word written in all caps and adds a special token for all caps (`xxcap`) in front of it
- `replace_maj`:: Lowercases a capitalized word and adds a special token for capitalized (`xxmaj`) in front of it
- `lowercase`:: Lowercases all text and adds a special token at the beginning (`xxbos`) and/or the end (`xxeos`)

Şimdi bunların kullanıldığı bir text için tokenization'a bakalım:

In [22]:
coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

Gördüğümüz gibi, burada olan şey şu verilen text, modelin dili daha rahat kavrayabilmesi için tokenize ediliyor. Bu tokenizasyon genelde her kelime için ayrı bir token üretmek üzerine işliyor ama bunun yanında daha bir çok rule ile aslında cümlede farklı yapıları da tokenize ediyor. Örneğin büyük harfle başlayan kelime için, önce büyük harf tokeni sonra kelime tokeni kullanılıyor vesaire. 

Eğer biz bu ekstra kurallarla tokenizasyonu yapmazsak, modelin dili öğrenmesi zorlaşacaktır, !!!!! için ayrı bir token, !! için ayrı bir token üretecek ve bu tokenler birbirinden tamamen bağımsız olacaktır, ancak biz ünlem tokeni ve repetition tokeni kullanırsak aslında bu tokenler arasındaki ilişki model tarafından çok daha kolay biçimde yakalanabilecektir.

Burada önemli bir nokta şu, yukarıda hiç belirtilmemiş ama tokenizer'ın asıl amacı aslında bir vocab oluşturmak. Yani evet tokenizer tüm corpus'u tokenlerle ifade etmeye çalışıyor ama bu süreçte kısıtlı sayıda token kullanmak zorunda, çünkü language model'in output'u bu token'lerden biri olacak. Language model verilen token sequence'ine karşılık next token'i bulmaya çalışacak.

Anladığım kadarıyla, word tokenizer için henüz ortada bir vocab falan yok. Şuanda tokenizer corpus'taki tüm kelimeler için token üretiyor, yani vocab size şuan infinite gibi düşün. WordTokenizer için vocab kısmı numericalization başlığı altında görülecek. Peki o halde burada neden unknown tokeni var nerede kullanılacak? Tam emin değilim büyük ihtimalle kullanılmayacaktır, her türlü kelime için bir kelime tokeni oluşturulacak çünkü. Unkown tokeni, vocab oluşturulunca mantıklı olacak, çünkü bu aşamada elimizdeki token sayısı kısıtlanacağı için, vocab içerisinde olmayan kelimeleri unknown ile temsil edeceğiz.

Şimdi subword tokenization'a bakalım:

### Subword Tokenization

Bir başka tokenization yöntemi **subword tokenization**'dır.  Word tokenization boşlukların cümle içerisindeki componentlerin etkili bir şekilde ayrımı için kullanıldığı varsayımına dayanır. Ancak durum her zaman bu değildir. Örneğin Çince de şu cümle:
我的名字是郝杰瑞  "Benim adım Jeremy Howard" demektir. Word tokenizer ile böyle bir cümleyi tokenize edemeyiz arada space yok. Japanese, Chinese gibi dillerde boşluk kullanılmaz aslında ortada well-defined bir kelime kavramı bile yoktur.

Bunun yanında Türkçe, Macarca gibi dillerde birçok subword'ün boşluk olmadan bir araya gelir ve içerisinde çok farklı anlamlar barındıran uzun kelimeler oluşturur. Tek bir kelime içerisinde, özne, zaman gibi bilgiler eklerle saklanabilir. 

İşte bu tip durumları daha iyi handle edebilmek için genelde subword tokenization kullanmak en iyisidir. Bu yöntem iki adımdan oluşur:

1. Öncelikle corpus of documents incelenir ve en sık görülen groups of letters tespit edilir. Bunlar bizim vocab'imiz olacak.
1. Daha sonra corpus'u bu vocab'i (subword units) kullanarak tokenize ederiz.

Bir örneğe bakalım. Corpus olarak için ilk 2k movie reviews kullanılacak.

In [10]:
#2000 tane file okundu içeriği txts listesi (L) içerisine atıldı.
txts = L(o.open().read() for o in files[:2000])

Yukarıda txts listesi içerisinde 2000 adet review tutuyor. Şimdi tokenizer'ımızı instantiate edeceğiz, daha sonra da tokenizer'ı train edeceğiz. 

WordTokenizer için text'leri doğrudan tokenizer içerisine veriyorduk ve sonucunda bir vocab oluşturulup bu vocab ile tüm text tokenize edilmiş oluyordu.

Subword tokenizer için ise bir SubwordTokenizer objesi yaratılıyor, burada vocab_sz parametresini veriyoruz, daha sonra da training gerekli. Yani mpdelin tüm dökümanları okuyup vocab'i oluşturacak common character sequences'i bulması gerek. Burada "character sequences" kelime olabileceği gibi kelime'den daha küçük parçalar da olabilir. Bu corpus'u okuyup vocab oluşturma işi yani training işi `setup` ile yapılır. 

Yakında göreceğimiz gibi "setup" methodu data processing pipeline'larında otomatik olarak çağırılan bir fastai methodudur. Ancak şimdilik herşeyi manuel yapıyoruz o yüzden bu methodu kendimiz çağırıyoruz. Aşağıdaki fonksiyon, subword tokenizer'ı verilen vocab size ile oluşturan ve sonra da bu eğitilmiş tokenizer içerisine txt'yi alır ve tokenize eder, sonra da ilk 40 tokeni birleştirip return eder.

In [11]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

Şimdi 2000 reviews'i alıp 1000 subword token ile tokenize etsin, 2000 reviews içerisinde en sık görülen 1000 token'i vocab'ine alacak.

In [12]:
subword(1000)

"▁The ▁worst ▁movie ▁I ' ve ▁ever ▁seen , ▁hand s ▁down . ▁It ▁is ▁t en ▁time s ▁more ▁a ▁ r i p - off ▁of ▁L ake ▁P la ci d ▁than ▁it ▁is ▁a ▁sequel ."

Fastai's subword tokenizer'ını kullanırken special character `▁` orjinal text'teki boşluk karakterini temsil eder.

Eğer 1000 yerine 200 token kullanırsak, daha küçük bir vocab kullanmış olacağız bu yüzden tokenler daha kısa karakterleri temsil edecek bu yüzden de bir cümleyi represent etmek için daha çok token gerekecek. Aşağıda txt'yi tokenize edip ilk 40 token'i bastırınca görüyoruz ki cümlenin daha kısa bir kısmını return ediyor. 

Bu gösterimde sanıyorum space'ler tokenleri ayrı ayrı görmemizi sağlamak için var, her tokenden sonra bir space var, "_" işareti ise tokenize edilmiş space.

In [13]:
subword(200)

"▁The ▁w or s t ▁movie ▁I ' ve ▁ e ver ▁s e en , ▁ h an d s ▁d o w n . ▁I t ▁is ▁ t en ▁ t i m es ▁mo re ▁a"

Eğer vocab'i büyütürsek, o halde ne olacak, vocab içerisine daha uzun karakter dizileri alabilecek, bunlar belki de doğrudan kelimeler olacak, çünkü vocab'de kaydedilecek yeri var. Common words ve ekler vocab'de oluşturulacak. Aşağıda görüyoruz ki bu kez token'lerin bir çoğu direkt kelimeleri temsil ediyor.

In [14]:
subword(10000)

"▁The ▁worst ▁movie ▁I ' ve ▁ever ▁seen , ▁hands ▁down . ▁It ▁is ▁ten ▁times ▁more ▁a ▁rip - off ▁of ▁Lake ▁Placid ▁than ▁it ▁is ▁a ▁sequel . ▁Director ▁David ▁F lo re s ▁clearly ▁did ▁not ▁go"

Subword vocab size seçimi bir tradeoff gerektirid. Daha büyük vocab size seçersek, daha uzun tokenler mümkün, cümleler daha az tokenle ifade edilebiliyor, bu da faster training, less memory, less state for the model to remember demek. Ancak buna karşın, larger embedding matrices demek, yani more data to learn.

Sonuçta subword tokenization kullanarak character tokenization (i.e., using a small subword vocab) ile word tokenization (i.e., using a large subword vocab) arasında kolayca geçiş yapabiliyoruz. Böylece her türlü dili language-specific algorithms design etmeden öğrenmemiz mümkün oluyoruz. 

Genomic sequence veya MIDI music notation gibi dilleri de öğrenmenin mümkün olduğunu unutma. İşte bu sebebplerden dolayı, subword tokenization'ın popülaritesi geçtiğimiz yıllarda tırmandı, ve most common tokenization approach'larından biri haline geldi.
 
Overall, subword tokenization provides a way to easily scale between character tokenization (i.e., using a small subword vocab) and word tokenization (i.e., using a large subword vocab), and handles every human language without needing language-specific algorithms to be developed. It can even handle other "languages" such as genomic sequences or MIDI music notation! For this reason, in the last year its popularity has soared, and it seems likely to become the most common tokenization approach (it may well already be, by the time you read this!).

Sonuç olarak tokenization adımında, verilen corpus için tokenizer'ımız en popüler subwords'leri kendine token olarak belirledi, artık bu token'leri kullanarak tokenizer'ımızın her türlü cümleyi token'ler ile ifade edebilmesini bekliyoruz. Bunun için daha önce görmediği kelimeleri veya yapıları da temsil edebilmesi gerek yani bir çeşit unknown tokeni de olduğunu kabul ediyorum.

Şimdi sıradaki adım, numericalization.

### Numericalization with fastai

**Numericalization** adımında vocab içerisindeki tokenleri integers ile map ederiz. Bu aşama bir `Categorical` variable'ı modele beslemek için yapılan hazırlık ile aynı.


1. Categorical variable'ın all possible levels'inin listesi yapılır. (Bu zaten yapıldı, THE VOCAB)
1. Vocab içerisindeki her level'i index'i ile değiştir. 

Yani bir nevi token'leri numbers ile değiştirmiş olduk, her integer'ın hangi tokene karşılık geldiğini de sakladıktan sonra hiçbir bilgi kaybetmemiş oluruz. Numeric tokenler ile başetmesi bizim için çok daha kolay olacaktır. Artık cümleleri sequences of integers şeklinde temsil edebiliriz. Daha sonra her bir tokeni bir embedding vector ile de map edeceğiz.


Başta gördüğümüz word-tokenized text için numericalization'a bakalım:

In [28]:
#txt'yi tokenize ettik, yani tokenlerle temsil ettik, ilk 31 tokeni print ediyoruz:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#176) ['xxbos','xxmaj','the','worst','movie','xxmaj','i',"'ve",'ever','seen',',','hands','down','.','xxmaj','it','is','ten','times','more','a','rip','-','off','of','xxmaj','lake','xxmaj','placid','than','it'...]


Yukarıda yapılan iş, verilen kısa corpus için (txt), tüm text'i tokenize etmek ve toks içerisine almak. Daha sonra bu tokenlerin bir kısmı print edildi.

Sıradaki adımda, tek bir review yani "txt" için tokenize işlemini yapmak yerine, 200 tane review için tokenization yapacağız, bunun için aşağıdaki gibi map fonksiyonu kullanılıyor. Burada txts[i] bize bir review verecek, toks200[i] ise o review'in tokenize edilmiş halini verecektir. 

In [29]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#176) ['xxbos','xxmaj','the','worst','movie','xxmaj','i',"'ve",'ever','seen'...]

Hatırlarsak, WordTokenizer için şuana kadar sadece tokenization işlemi yaptık, bir fitting vesaire yapılmadı yani bir vocab oluşturulmadı, önüne gelen her türlü cümleyi tokenize etti.

Vocab oluşturmak için önce bir `Numericalize` objesi instantiate edilir. Daha sonra bu num objesinin `setup` methodu tokenize edilmiş corpus ile çağırılır. İşte bu aşamada vocab yaratılmış olur.

Yani ilk önce tüm corpus tokenize ediliyor, daha sonra bu tokenized corpus numericalize objesi içerisine alınarak, bu corpustan bir vocab oluşturuluyor.

Aşağıda bunu görüyoruz, vocab'i oluşturduktan sonra ilk 20 tokenini de görüntüledik:

In [35]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#2144) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the',',','.','and','a','of','to','is','in','it','i'...]"

Vocab'in en başında, special rules token'lerin yeraldığını görebiliriz, daha sonra kelimeler frequency order'a göre dizilir (sanırım azalan sırada). 

Burada vocab oluştururken bir size vesaire girmedik, o yüzden default değerler kullanıldı. Bu default değerler şöyle: `min_freq=3, max_vocab=60000`. Bu şu demek, vocab size = 60k yani en common 60k word token olarak tutulacak, bunun dışındaki kelimeler `xxunk` tokeni ile temsil edilecek. Vocab size'ın çok büyük olmaması çok büyük embedding matrix'lerden kaçınmak için gereklidir, çok büyük embedding matrices training süresini yavaşlatacak ve çok fazla memory kullanacaktır. 

Ayrıca `min_freq=3` de bir kelimenin kendine has bir tokeni olabilmesi için en az 3 kez occur olması gerektiği anlamına gelir, üçten az görülen kelimeler yine `xxunk` olarak ele alınır. 

Vocab'i oluşturduktan sonra, num objesine verilen tokenized bir set, vocab kullanılarak number sequences olarak da temsil edilebilir. Aşağıda görüldüğü gibi, num ojesi içerisine tokenized bir text veriliyor, ve sonuç olarak bu tokenized text vocab kullanılarak numericalize edilmiş oluyor.


In [39]:
nums = num(toks)[:20]; nums

TensorText([  2,   8,   9, 310,  27,   8,  19, 218, 158, 141,  10,   0, 229,  11,
          8,  18,  16, 550, 299,  66])

Artık herhangi bir cümleyi önce tokenize edip, daha sonra da bu tokenized version'ı num objesine besleyerek numericalized version of this sentence'ı elde edebiliriz. Bu numericalized version'ı modellerde embedding layers'a input olarak kullanabiliriz.

Ayrıca unutma ki numeric cümleyi vocab kullanarak tekrar tokenized cümleye çevirebiliriz:

In [40]:
' '.join(num.vocab[o] for o in nums)

"xxbos xxmaj the worst movie xxmaj i 've ever seen , xxunk down . xxmaj it is ten times more"

Artık text'leri numericalize edebiliyoruz, hafif hafif language model eğitmeye doğru ilerliyoruz. Öncelikle text'leri batches haline nasıl getireceğiz ona bakalım:

### Putting Our Texts into Batches for a Language Model

Images ile çalışırken, hepsini aynı boyutlara resize etmiştik ve daha sonra mini-batch'lerde stack etmiştik. Böylece her bir minibatch'i bir tensor olarak ifade edebiliyorduk. 

Burada işler biraz daha farklı olacak, çünkü text'leri images gibi resize etmemiz kavramsal olarak mümkün değil. Burada batch'lerin sıralı olması önemli, yani current batch bir önceki batch'in bittiği yerden başlıyor olmalı çünkü language model text'i sıralı şekilde okumalı. Ancak elbette review'lerin sırası çok önemli değil, review'ler karılabilir sonucunda oluşan kocaman corpus'u biz batch'lere böleceğiz bunu böldüğümüzde batch'ler sırasıyla birbirini tamamlamalı.

Örneğin elimizde aşağıdaki gibi bir text olsun bunu corpus'umuz gibi düşünebiliriz.

> : In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while.

Sonuçta tokenization process bu corpusu alıp tokenize versiyonu return edecek:

> : xxbos xxmaj in this chapter , we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface . xxmaj first we will look at the processing steps necessary to convert text into numbers and how to customize it . xxmaj by doing this , we 'll have another example of the preprocessor used in the data block xxup api . \n xxmaj then we will study how we build a language model and train it for a while .

Yukarıda gördüğümüz gibi corpus'umuz 90 token'le represente ediliyor. Let's say we want a batch size of 6. We need to break this text into 6 contiguous parts of length 15.

In [46]:
#hide_input
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


Yukarıdaki örnekte corpus'u 6 batch'e ayırmış bulunduk ancak bu yöntem scale etmez. Çünkü bu örnekte corpusumuz sadece 90 token ile represente edilebildiği için batch size 6 olduğunda cümleler 15 tokenden oluşuyordu ancak gerçekte IMBD review corpus'unda 90 yerine milyonlarca token söz konusu ben bu yaklaşımla corpus'u 6'ya bölersem tek bir data point 15 yerine yüzbinlerce tokenden oluşmak zorunda kalacak, bu yaklaşım sıkıntılı.

Bu yüzden bu array'i fixed sequence length ile daha küçük subarray'e bölmeliyiz. Bu subarrays için order'ı korumak önemli çünkü kullanılacak model sıradaki kelimeyi tahmin ederken daha önce okuduklarını da baz alarak bir karar verecek.

15 length'li 6 batch'den oluşan örneğe geri dönersek, sequence length = 5 seçersek, bu şu demek ilk olarak aşağıdaki array beslenecek:

In [47]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build


Daha sonra bu:

In [48]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

",",we,will,go,back
chapter,1,and,dig,deeper
the,processing,steps,necessary,to
xxmaj,by,doing,this,","
the,data,block,xxup,api
a,language,model,and,train


En son da bu:

In [49]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


Movie reviews datasetine geri dönersek, ilk adım individual texts'i concatenate ederek bir stream elde etmek. Her epoch başında reviews shuffle edilerek random bir review order elde ederiz, daha sonra concatenation ile tüm reviewleri random order ile içeren stream elde edilir.

Daha sonra bu stream'e certain number of batches'e böleriz bu da bizim **batch size**'ımız olacak. Örneğin stream 50k tokens içeriyorsa, batch size = 10 ise elimizde 10 adet 5k tokenden oluşan mini-streams olacaktır. Burada her mini-stream'in ordered olduğunu unutma, yani ilk mini-stream 1-5000. tokenleri içerirken 2. mini-stream 5001-10000. tokenleri içerir.

Ayrıca `xxbos` tokeni sayesinde model stream'leri okurken, yeni review'in nerede başladığını biliyor olacak.

So to recap, at every epoch we shuffle our collection of documents and concatenate them into a stream of tokens. We then cut that stream into a batch of fixed-size consecutive mini-streams. Our model will then read the mini-streams in order, and thanks to an inner state, it will produce the same activation whatever sequence length we picked.

This is all done behind the scenes by the fastai library when we create an `LMDataLoader`. We do this by first applying our `Numericalize` object to the tokenized texts:

In [61]:
nums200 = toks200.map(num)

and then passing that to `LMDataLoader`:

In [62]:
dl = LMDataLoader(nums200)

İlk batch'i grab edelim:

In [64]:
x,y = first(dl)
x.shape,y.shape

((64, 72), (64, 72))

Dataset'in ilk row'una bakalım, first review'in başlangıcı olmalı:

In [54]:
' '.join(num.vocab[o] for o in x[0][:20])

"xxbos xxmaj the worst movie xxmaj i 've ever seen , xxunk down . xxmaj it is ten times more"

Buna karşılık gelen dependent token ise 1 offset ile aynı text:

In [55]:
' '.join(num.vocab[o] for o in y[0][:20])

"xxmaj the worst movie xxmaj i 've ever seen , xxunk down . xxmaj it is ten times more a"

Böylelikle datamıza tüm preprocessing adımlarını uygulamış olduk artık bir text classifier train edebiliriz.

## Training a Text Classifier